In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from registration_utilities import *
from annotation_utilities import *
from metadata import *
from data_manager import *

Setting environment for Precision WorkStation


No vtk


In [3]:
from aligner_v3 import Aligner

In [4]:
lauren_brains = ['LM42_LM37', 'LM94_LM96_LM25', 'LM38', 'LM30new', 'LM27', 'LM37', 'LM22', 'LM32', 'LM17', 'LM48', 'LM31', 'LM95', 'LM41', 'LM84', 'LM40new', 'LM86', 'LM54', 'LM46']

In [ ]:
for brain_name in lauren_brains:
# for brain_name in ['LM42_LM37']:

    brain_f_spec = dict(name=brain_name, vol_type='annotationAsScore', structure='SNR_L', resolution='10.0um')
    brain_m_spec = dict(name='atlasV7', vol_type='score', structure='SNR_L', resolution='10.0um')
    alignment_spec = dict(stack_m=brain_m_spec, stack_f=brain_f_spec, warp_setting=7)

    aligner_parameters = generate_aligner_parameters_v2(alignment_spec=alignment_spec, 
                                                        structures_m=['SNR_L'],
                                                       fixed_structures_are_sided=True,
                                                       fixed_surroundings_have_positive_value=True,
                                                       fixed_use_surround=True)
    
    aligner = Aligner(aligner_parameters['volume_fixed'], 
                      aligner_parameters['volume_moving'], 
                      labelIndexMap_m2f=aligner_parameters['label_mapping_m2f'],
                     invalid_voxel_penalty=0.)

    aligner.set_centroid(centroid_m='structure_centroid', 
                         centroid_f='structure_centroid', 
#                          indices_m=[aligner_parameters['structure_to_label_moving']['SNR_L']]
                        )

    aligner.compute_gradient(smooth_first=True)

    aligner.set_label_weights(label_weights=aligner_parameters['label_weights_m'])

    _, _ = aligner.optimize(tf_type=aligner_parameters['transform_type'], 
                                 max_iter_num=5000,
                                 history_len=100, 
                                 terminate_thresh_trans=.01,
                                terminate_thresh_rot=0.01,
                                 full_lr=np.array([1,1,1,.01,.01,.01]),
                                )

    plot_alignment_results(traj=aligner.Ts, scores=aligner.scores, select_best='max_value')
    
    DataManager.save_alignment_results_v3(aligner=aligner, 
                                          select_best='last_value',
                                          alignment_spec=alignment_spec,
                                          )